In [1]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [2]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [3]:
def error_detection(theta, phi):
    @squin.kernel 
    def parameterized_error_detection():
        MSD_enc = MSD_encoding(theta, phi)
        z_ancilla = MSD_encoding((np.pi)/2, 0) # Injected state is |+>
        z_measurement = []

        for i in range(7):
            squin.cx(MSD_enc.q[i], z_ancilla.q[i])
            z_measurement.append(squin.measure(z_ancilla.q[i]))
            # print(z_measurement[i])
        
        x_ancilla = MSD_encoding(0, 0) # Injected state is |0>
        x_measurement = []
        for i in range(7):
            squin.cx(x_ancilla.q[i], MSD_enc.q[i])
            squin.h(x_ancilla.q[i])
            squin.measure(x_ancilla.q[i])
            x_measurement.append(squin.measure(x_ancilla.q[i]))
            # print(x_measurement[i])
    return parameterized_error_detection

In [4]:
err_det_circ = bloqade.tsim.Circuit(error_detection(0,0))
err_det_circ.diagram(height=600)

('callee must be a method type, got function', 'when verifying the following statement', ' `func.Call` at\n', '    \x1bfunc\x1b\x1b.\x1b\x1bcall\x1b %0(%1, %2)\x1b : \x1b\x1b!\x1b\x1bBottom\x1b\x1b maybe_pure=False\x1b\n')


In [5]:
def MSD_encoding(theta, phi,basis="z"):
    @squin.kernel
    def z_MSD_encoding(q_subset): # MSD encoding of Z-basis ancilla (|+>); equivalent to the line z_ancilla = MSD_encoding((np.pi)/2, 0) before
        squin.u3((np.pi)/2,0,0,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q_subset[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q_subset[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
    @squin.kernel
    def parameterized_MSD_encoding(): # MSD encoding of both injected state and Z-basis ancilla
        q = squin.qalloc(14)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
        z_MSD_encoding([q[7], q[8], q[9], q[10], q[11], q[12], q[13]]) # Adding the Z-basis ancilla encoding
    
    return parameterized_MSD_encoding

In [6]:
MSD_enc = MSD_encoding(0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)

In [37]:
def MSD_encoding_plus_ancilla(theta, phi,lam,basis="z"):
    @squin.kernel
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        q=squin.qalloc(14)
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi) # Z-basis ancilla encoding

        squin.cx(q[0],q[7])
        squin.cx(q[1],q[8])
        squin.cx(q[2],q[9])
        squin.cx(q[3],q[10])
        squin.cx(q[4],q[11])
        squin.cx(q[5],q[12])
        squin.cx(q[6],q[13])

        z_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        # z_measurement = []
        # z_measurement.append(squin.measure(q[7]))
        # z_measurement.append(squin.measure(q[8]))
        # z_measurement.append(squin.measure(q[9]))
        # z_measurement.append(squin.measure(q[10]))
        # z_measurement.append(squin.measure(q[11]))
        # z_measurement.append(squin.measure(q[12]))
        # z_measurement.append(squin.measure(q[13]))

        squin.broadcast.reset([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=0,phi=0,lam=0) # X-basis ancilla encoding

        squin.cx(q[7],q[0])
        squin.cx(q[8],q[1])
        squin.cx(q[9],q[2])
        squin.cx(q[10],q[3])
        squin.cx(q[11],q[4])
        squin.cx(q[12],q[5])
        squin.cx(q[13],q[6])

        squin.broadcast.h([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        x_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        # x_measurement = []
        # x_measurement.append(squin.measure(q[7]))
        # x_measurement.append(squin.measure(q[8]))
        # x_measurement.append(squin.measure(q[9]))
        # x_measurement.append(squin.measure(q[10]))
        # x_measurement.append(squin.measure(q[11]))
        # x_measurement.append(squin.measure(q[12]))
        # x_measurement.append(squin.measure(q[13]))

    return circ

In [38]:
MSD_enc = MSD_encoding_plus_ancilla(1,0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)